# Report 5 Summary

1. Download representitive sequences from Gencode gencode_repr.fasta
2. Perform exact alignment with all the SRR11015356 reads against the gencode_repr.fasta 
3. Perform exact alignment with the before75_after75 unitigs file against gencode_repr.fasta
4. Generate coverage graphs using the two generated bam files from the previous steps
